In [1]:
import time
from selenium import webdriver

In [2]:
from selenium.webdriver import ChromeOptions
options = ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome("C:/chromedriver", options=options)

C:\Users\ameli\AppData\Local\Temp/ipykernel_15184/2632285444.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:/chromedriver", options=options)


In [3]:
url = "https://www.youtube.com/watch?v=XrIaJayElnw"
driver.get(url)
time.sleep(3)

In [4]:
last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
driver.execute_script("window.scrollTo(0, 500);") # 댓글영역이 조금 보이도록 스크롤
time.sleep(1)

In [5]:
driver.find_element_by_css_selector("div#icon-label").click() # 정렬 기준 클릭
time.sleep(1)
driver.find_element_by_css_selector("a[tabindex='-1'] div.item").click() # 정렬 기준 > 최근 날자순 클릭

C:\Users\ameli\AppData\Local\Temp/ipykernel_15184/3373272336.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector("div#icon-label").click() # 정렬 기준 클릭
C:\Users\ameli\AppData\Local\Temp/ipykernel_15184/3373272336.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector("a[tabindex='-1'] div.item").click() # 정렬 기준 > 최근 날자순 클릭


In [6]:
print(last_page_height)

2201


In [7]:
import time
page_count = 0
while True:
    scroll_script = "window.scrollTo(0, document.documentElement.scrollHeight);"
    driver.execute_script(scroll_script)
    time.sleep(3)
    scroll_height = "return document.documentElement.scrollHeight"
    new_page_height = driver.execute_script(scroll_height)
    
    if new_page_height==last_page_height:
        break
    last_page_height = new_page_height   
    
    if page_count > 10:
        break
    else:
        page_count += 1


In [8]:
html_source = driver.page_source
driver.close()

In [9]:
from bs4 import BeautifulSoup

In [10]:
print(html_source[:100])

<html style="font-size: 10px;font-family: Roboto, Arial, sans-serif;" lang="ko-KR" system-icons="" t


In [11]:
soup = BeautifulSoup(html_source, "html.parser")

In [12]:
comments = soup.select("yt-formatted-string#content-text")
len(comments)

194

In [13]:
comments[0].text

'헬륨을 넣은 지수 목소리 너무 귀여웠어요 🤩'

In [14]:
users = soup.select("a#author-text > span") # div#header-author a > span
len(users)

194

In [15]:
users[0].text.strip() # strip는 텍스트 앞/뒤 공백 제거

'ramesh g'

In [16]:
user_list = [ user.text.strip() for user in users]

In [17]:
comment_list = [ comment.text.strip() for comment in comments]

In [18]:
import pandas as pd
import numpy as np
comment_df = pd.DataFrame(np.c_[user_list, comment_list], columns=["user", "comment"])

In [19]:
comment_df.head()

,user,comment
0,ramesh g,헬륨을 넣은 지수 목소리 너무 귀여웠어요 🤩
1,Ali Kaleli,Jensoo and chaelisaa
2,Ali Kaleli,roséé yeaah
3,Ali Kaleli,Jisoo unnie 😅
4,Ali Kaleli,Oh..old


In [20]:
comment_df.to_csv("comment.csv", encoding="utf-8-sig", index=False)